In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import *
from constants import *
from imports import *
from loaders import *
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

('1.13.1', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'23b345k5'

In [3]:
wandb.init(id='2njo5bon', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) 
m.train()
sum([torch.numel(p) for p in m.parameters()]) / 1000

30484.11

In [5]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(m, wandb=wandb)

CPU times: user 1 ms, sys: 352 µs, total: 1.35 ms
Wall time: 1.14 ms


In [6]:
m = try_load_state_dict(m, torch.load(f"{BESPOKE_ROOT}/models/m.torch"))

In [7]:
model_stem = "2.17"

In [8]:
%%time

rnn_only = True #True
rnn_is_first = True #False

if rnn_only: # m cnn backbone needs to be all loaded up and ready

    if rnn_is_first:
        freeze_model(m, True)
        freeze_model(m.rnn, False)
        m.hidden_init.requires_grad = True
        m.cell_init.requires_grad = True
    else:
        freeze_model(m.backbone, True)
        
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 21 #7 # these have to be multiple of Zloader's internal bptt, currently 3 for bs 32 or 7 for bs 16
    seqlen = bptt*10 #5 #15 
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    n_workers = 5 if seqlen > 100 else 8 # maxxing out ram here w higher bs. Was 8 w bs 16
    # fewer than five workers we start to have to wait too long for chunk. More is better here.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = 0 #.9 TODO temp disabled
    n_workers = 8
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
    
lr = 3e-4
m.use_rnn = rnn_only

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been depreca

waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
CPU times: user 2.24 s, sys: 17.8 s, total: 20.1 s
Wall time: 1min 29s


In [9]:
opt = torch.optim.Adam(m.parameters(), lr=lr)
#opt = torch.optim.AdamW(m.parameters(), lr=3e-4, weight_decay=.001)

In [10]:
m.backbone.training, m.training

(False, True)

In [11]:
[(n,p.requires_grad) for (n,p) in m.named_parameters()]

[('hidden_init', True),
 ('cell_init', True),
 ('backbone.conv_stem.weight', False),
 ('backbone.bn1.weight', False),
 ('backbone.bn1.bias', False),
 ('backbone.blocks.0.0.conv_dw.weight', False),
 ('backbone.blocks.0.0.bn1.weight', False),
 ('backbone.blocks.0.0.bn1.bias', False),
 ('backbone.blocks.0.0.se.conv_reduce.weight', False),
 ('backbone.blocks.0.0.se.conv_reduce.bias', False),
 ('backbone.blocks.0.0.se.conv_expand.weight', False),
 ('backbone.blocks.0.0.se.conv_expand.bias', False),
 ('backbone.blocks.0.0.conv_pw.weight', False),
 ('backbone.blocks.0.0.bn2.weight', False),
 ('backbone.blocks.0.0.bn2.bias', False),
 ('backbone.blocks.0.1.conv_dw.weight', False),
 ('backbone.blocks.0.1.bn1.weight', False),
 ('backbone.blocks.0.1.bn1.bias', False),
 ('backbone.blocks.0.1.se.conv_reduce.weight', False),
 ('backbone.blocks.0.1.se.conv_reduce.bias', False),
 ('backbone.blocks.0.1.se.conv_expand.weight', False),
 ('backbone.blocks.0.1.se.conv_expand.bias', False),
 ('backbone.block

In [12]:
"""%%time
m.model_stem = model_stem
rw_evaluator.evaluate()"""

'%%time\nm.model_stem = model_stem\nrw_evaluator.evaluate()'

In [13]:
for n, mm in m.backbone.named_modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .99 #.4

In [14]:
trainer = Trainer(dataloader_trn, m, model_stem=model_stem, opt=opt, wandb=wandb, 
                  rw_evaluator=rw_evaluator, rnn_only=rnn_only)

In [15]:
trainer.reload_state()

Loading model from /home/beans/bespoke/models/m.torch. 
Last modified 3 min ago.
Loading opt from /home/beans/bespoke/models/opt.torch. 
Last modified 3 min ago.
Currently at epoch 22.


In [16]:
copy_cnn_params = False
if copy_cnn_params:
    m.copy_cnn_params_to_rnn() # when first trn rnn, has to be after reload_state.
    print("copied")
    
if rnn_only:
    trainer.loss_manager.update_emas = False
    trainer.loss_manager.loss_emas['te'] = .002 # manual. Bc cnn doesn't cache this.

In [17]:
trainer.loss_manager.update_emas

False

In [18]:
trainer.loss_manager.loss_weights

{'has_stop': 0.1,
 'stop_dist': 0.1,
 'has_lead': 0.1,
 'lead_dist': 0.1,
 'lead_speed': 0.02,
 'lane_width': 0.02,
 'dagger_shift': 0.05,
 'rd_is_lined': 0.02,
 'left_turn': 0.01,
 'right_turn': 0.01,
 'td': 0.03,
 'pitch': 0.1,
 'yaw': 0.1,
 'unc': 0.1,
 'te': 0.05,
 'wp_angles': 1,
 'wp_angles_i': 0.5,
 'wp_curvatures': 0.2,
 'wp_curvatures_i': 0.1,
 'wp_headings': 0.02,
 'wp_headings_i': 0.01,
 'wp_rolls': 0.05,
 'wp_rolls_i': 0.025,
 'wp_zs': 0.02,
 'wp_zs_i': 0.01}

In [19]:
trainer.loss_manager.loss_emas

{'has_stop': 0.009303476897794368,
 'stop_dist': 0.0024502977352331263,
 'has_lead': 0.005678485629850111,
 'lead_dist': 0.001069278861559114,
 'lead_speed': 0.009430690396974376,
 'lane_width': 0.047450313235093605,
 'dagger_shift': 0.02728865822736125,
 'rd_is_lined': 0.001319377378716812,
 'left_turn': 0.0012632710253530598,
 'right_turn': 0.0005388626623519055,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.153600983920089,
 'te': 0.002,
 'wp_angles': 0.09190313942485771,
 'wp_angles_i': 0.31636149843256006,
 'wp_curvatures': 0.7550056203429856,
 'wp_curvatures_i': 1.4115027495777035,
 'wp_headings': 0.1158619262927107,
 'wp_headings_i': 0.28565145688222876,
 'wp_rolls': 3.8420652287183055,
 'wp_rolls_i': 0.015127179046717893,
 'wp_zs': 4.422409783316988,
 'wp_zs_i': 0.0953607479912514}

In [ ]:
trainer.train()

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


wp_angles_i tensor(0.0178, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.0767, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(0.3418, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0254, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0578, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0599, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1448, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.4827, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(2.0430, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(4.2617, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te tensor(0.0029, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
has_stop tensor(0.0015, dev

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning:

None of the inputs have requires_grad=True. Gradients will be None



wp_angles_i tensor(0.1594, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.5098, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(1.6309, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0203, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0341, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0476, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.0968, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.5591, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(1.5156, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(4.8516, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te tensor(0.0032, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
has_stop tensor(0.0147, dev

left_turn tensor(0.0043, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
right_turn tensor(5.0068e-06, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
unc tensor(67.4375, device='cuda:0', dtype=torch.float16)
wp_angles_i tensor(0.0764, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.1731, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(2.7051, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0693, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.2871, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.1082, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1646, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.5420, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(5.3789, device='c

In [20]:
AUX_TARGET_PROPS

['has_stop',
 'stop_dist',
 'has_lead',
 'lead_dist',
 'lead_speed',
 'dagger_shift',
 'lane_width',
 'rd_is_lined',
 'left_turn',
 'right_turn']

In [20]:
trainer.loss_manager.loss_emas

{'has_stop': 0.6581827152218972,
 'stop_dist': 0.1914074304465102,
 'has_lead': 0.6045025000545183,
 'lead_dist': 0.18079276826857715,
 'lead_speed': 0.8834656769419045,
 'lane_width': 4.096574170043749,
 'dagger_shift': 2.512469867257369,
 'rd_is_lined': 0.10581200184235365,
 'left_turn': 0.08856740587054233,
 'right_turn': 0.07401701428385941,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 117.34792707051348,
 'te': 0.002,
 'wp_angles': 0.0859911379087128,
 'wp_angles_i': 4.15318218310992,
 'wp_curvatures': 0.7827851002866839,
 'wp_curvatures_i': 17.866540970943753,
 'wp_headings': 0.12228421566478304,
 'wp_headings_i': 2.901144808827211,
 'wp_rolls': 4.211842307760256,
 'wp_rolls_i': 0.13459417300430104,
 'wp_zs': 5.043518468202303,
 'wp_zs_i': 1.1309414719989206}

In [20]:
m.hidden_init, m.cell_init

(Parameter containing:
 tensor([[[ 0.0015, -0.0488, -0.0292,  ..., -0.0560, -0.0283,  0.0664]]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[[-0.3869,  0.0330,  0.0708,  ..., -0.0204,  0.3181, -0.2530]]],
        device='cuda:0', requires_grad=True))

In [21]:
m.training, m.backbone.training 

(True, False)

In [22]:
set_trainer_should_stop(False)

In [28]:
{k:trainer.loss_manager.loss_emas["wp_angles"]/trainer.loss_manager.loss_emas[k] for k,v in trainer.loss_manager.loss_emas.items()}

{'has_stop': 29.734797363340526,
 'stop_dist': 87.87280383313023,
 'has_lead': 40.201898525296386,
 'lead_dist': 127.76564504863305,
 'lead_speed': 0.34846566491395553,
 'lane_width': 3.809900606793288,
 'dagger_shift': 4.134576796863424,
 'rd_is_lined': 16457.381372040072,
 'left_turn': 125.55969474655697,
 'right_turn': 283.74133223646584,
 'td': 0.015300805875045518,
 'pitch': 0.015300805875045518,
 'yaw': 0.015300805875045518,
 'unc': 0.11512757866420895,
 'te': 61.54656219577027,
 'wp_angles': 1.0,
 'wp_angles_i': 0.240031243936724,
 'wp_curvatures': 0.16382346075192508,
 'wp_curvatures_i': 0.003522151816451157,
 'wp_headings': 0.5183519273228973,
 'wp_headings_i': 0.005565440760627469,
 'wp_rolls': 0.027721036050055964,
 'wp_rolls_i': 8.663331955712781,
 'wp_zs': 0.00020285505128895626,
 'wp_zs_i': 0.0035411505624853203}

In [23]:
{'has_stop': 0.005145757574225003,
 'stop_dist': 0.001741244754645775,
 'has_lead': 0.0038059908701619494,
 'lead_dist': 0.0011975680840668381,
 'lead_speed': 0.4390907746627964,
 'lane_width': 0.04016064316156499,
 'dagger_shift': 0.0370069456362572,
 'rd_is_lined': 9.297229935401817e-06,
 'left_turn': 0.0012186080816722507,
 'right_turn': 0.0005392519219686348,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.329030459302299,
 'te': 0.002486053701322257,
 'wp_angles': 0.15300805875045517,
 'wp_angles_i': 0.6374505928519476,
 'wp_curvatures': 0.9339813604728601,
 'wp_curvatures_i': 1.44164213359285,
 'wp_headings': 0.2951818073498582,
 'wp_headings_i': 1.49253209788985,
 'wp_rolls': 5.519564942456264,
 'wp_rolls_i': 0.017661571729288113,
 'wp_zs': 754.2728553133405,
 'wp_zs_i': 43.208571917673005


import sys
sys.getsizeof(trainer.loss_manager.loss_emas)

1176

In [16]:
torch.save(trainer.model.state_dict(), f"{BESPOKE_ROOT}/models/m{trainer.model_stem}_e{trainer.current_epoch}.torch")

In [16]:
m.training

Error in callback <function _WandbInit._resume_backend at 0x7f62ef023160> (for pre_run_cell):


Exception: The wandb backend process has shutdown

True

Error in callback <function _WandbInit._pause_backend at 0x7f62e3b863a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
trainer.opt.param_groups[0]['lr'] = 3e-4

In [28]:
snr

0.15067877149872744



 {'logistical/obs_consumed_per_second': 38.2265625, 'logistical/obs_generated_per_second': 85.1328125, 'logistical/slowest_runner_obs_per_sec': 5.3125, 'logistical/data_consumption_ratio': 0.45004667, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00400971, 'timing/model forward': 0.19439457, 'timing/calc losses': 0.00539116, 'timing/backwards': 1.11790375, 'timing/get worst': 0.00264816, 'timing/logging': 0.01570235, 'timing/calc timing': 0.01359244, 'timing/trn update': 1.35364565, 'wp_angles': 0.01003864, 'wp_curvatures': 0.04547036, 'wp_headings': 0.01048879, 'wp_rolls': 0.52840137, 'wp_zs': 0.09037566, 'has_stop': 0.01459547, 'stop_dist': 0.00825622, 'has_lead': 0.01966284, 'lead_dist': 0.00662124, 'lead_speed': 16.11909485, 'dagger_shift': 0.14152849, 'lane_width': 0.12975717, 'rd_is_lined': 0.01347083, 'pitch': 4.037e-05, 'yaw': 4.98e-05, 'unc': 1.57641983, 'logistical/max_param': 32.69097137, 'logistical/lr': 0.0006, 'logistical/mins_since_slowest

In [9]:
%%time

rnn_only = True

if rnn_only: # m cnn backbone needs to be all loaded up and ready
    #freeze_model(m, True)
    #unfreeze_part_of_model(m, "rnn")
    freeze_model(m.backbone, True)
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 14 #35 #8 #7
    seqlen = bptt*5 # needs to be mult of 7
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = .9
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug)

Launching 3 loader workers


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated.

Got first chunk
CPU times: user 1.1 s, sys: 5.72 s, total: 6.83 s
Wall time: 45.2 s


Process Process-3:
Process Process-4:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 133, in make_chunks
    time.sleep(.1)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 1

In [25]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
